
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  27.10.2023

--- 

# VISUM Model Preprocessing

<!--Notebook description and usage information-->
This notebook is to process the raw VISUM traffic model export and convert the data into gpkg files.

In [1]:
import sys

import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../../utils/')
import data_paths

In [2]:
# import visum data
visum_link_path = data_paths.VISUM_FOLDER_PATH + "analyse_2019_link.SHP"
visum_zone_path = data_paths.VISUM_FOLDER_PATH + "analyse_2019_zone.SHP"

visum_links = gpd.read_file(visum_link_path)
visum_zones = gpd.read_file(visum_zone_path)

print(f"Coordinate reference system of VISUM link data: {visum_links.crs}")
print(f"Coordinate reference system of VISUM zones data: {visum_zones.crs}")

Coordinate reference system of VISUM link data: EPSG:25832
Coordinate reference system of VISUM zones data: EPSG:25832


In [3]:
def hbefa_road_type(visum_typeno) -> str: 
    """returns hbefa compatible road type for all road types in the visum traffic model of Munich.
    The information below was retrieved from the official documentation of the VIUSM model and 
    matched to HBEFA types by intuition and exploration.
    
    0-10: gesperrte und fiktive Strecken sowie Fuß- und Radwege
    100-199: Autobahnen
    200-299: Bundesstraßen
    300-399: Staatsstraßen
    400-499: Kreisstraßen
    500-599: Gemeindestraßen
    600-699: Hauptstraßen (innerorts)
    700-799: Erschließungsstraßen
    800-899: Mittlerer Ring
    900-999: ÖV Strecken

    Args:
        visum_typeno (string object): defines the road type in the VISUM model

    Returns:
        str: HBEFA compatible road type
    """
    
    int_typeno = int(visum_typeno)
    
    if 0 <= int_typeno <=10: 
        return 'none'
    if 100 <= int_typeno <=199: 
        return 'Motorway-Nat'
    if 200 <= int_typeno <=299: 
        return 'TrunkRoad/Primary-National'
    if 300 <= int_typeno <=399: 
        return 'TrunkRoad/Primary-National'
    if 400 <= int_typeno <=499: 
        return 'Distributor/Secondary'
    if 500 <= int_typeno <=599: 
        return 'Distributor/Secondary'
    if 600 <= int_typeno <=699: 
        return 'Local/Collector'
    if 700 <= int_typeno <=799: 
        return 'Access-residential'
    if 800 <= int_typeno <=899: 
        return 'TrunkRoad/Primary-City'
    if 900 <= int_typeno <=999: 
        return 'none'
    else: 
        return 'none'

In [4]:
# add road type to visum link data
visum_links['road_type'] = visum_links.apply(lambda row: hbefa_road_type(row['TYPENO']), axis = 1)

# rename rows
visum_links.rename(columns={
    'SUMME_KFZ':'dtv_KFZ',
    'NO':'road_link_id',
    'BEL_PKW':'dtv_PKW',
    'BEL_LKW_K':'dtv_LCV',
    'CAPPRT':'hour_capacity',
    'V0PRT':'speed'
}, inplace = True)

# aggregate heavy vehicle rows
visum_links['dtv_HGV'] = visum_links[['BEL_LKW_M','BEL_LKW_G']].sum(axis = 1)

# Save preprocessed files

In [14]:
links_columns_to_keep = ['road_link_id', 'road_type', 'hour_capacity', 'speed', 
                   'dtv_KFZ', 'dtv_PKW', 'dtv_LCV', 'dtv_HGV', 'SLOPE', 'geometry']

visum_links[links_columns_to_keep].to_file(data_paths.VISUM_FOLDER_PATH+'visum_links.gpkg', driver='GPKG')

In [19]:
zones_columns_to_keep = ['qv_pkw', 'qv_lkw_k', 'geometry']
visum_zones[zones_columns_to_keep].to_file(data_paths.VISUM_FOLDER_PATH+'visum_zones.gpkg', driver='GPKG')